In [1]:
from jira import JIRA
import getpass
import os
import sys
import json
import argparse
import logging
from jira import JIRA
from dotenv import load_dotenv
import csv
from requests.auth import HTTPBasicAuth
from jira.exceptions import JIRAError
from datetime import datetime
import blessed
from blessed import Terminal
import logging
from jira import JIRAError
import requests
from ipywidgets import Button, VBox, Text
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, clear_output
jira_url = "https://jirasimplelib.atlassian.net"
user = "rimsha.ashfaq@verituslabs.com"
api_token = "ATATT3xFfGF0Nnf69USKMyeqvq0C4X8fxZc2v4Sn5F4VmxYIjEUjtuik2bqy-abJDodOiNCrakl5Ae5R1U-FlvE5AfNJ3b2ZGOprGWJ3GsEnMilU6Aff32m4xsrWhsQdsgQCwrtAPKRZnNU2DSgvvFlep3Twrd9vvZraD2mlW6aeVp_Q2SG3YT0=848384A6"
# jira = JIRA(server=jira_url, basic_auth=(user, api_token))
try:
    # Create a JIRA client instance
    jira = JIRA(basic_auth=(user, api_token), options={'server': jira_url})

    # Example: Retrieve details about the current user
    myself = jira.myself()
    print(f"My username: {myself.name}, Display name: {myself.displayName}")
    
except Exception as e:
    print(f"Error: {e}")



Error: JiraError HTTP 401 url: https://jirasimplelib.atlassian.net/rest/api/2/myself
	text: Client must be authenticated to access this resource.
	
	response headers = {'Date': 'Mon, 08 Jul 2024 11:03:22 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Content-Length': '53', 'Server': 'AtlassianEdge', 'Timing-Allow-Origin': '*', 'X-Arequestid': '713013dee4090481bdceb23a30132ab1', 'X-Seraph-Loginreason': 'AUTHENTICATED_FAILED', 'Cache-Control': 'no-cache, no-store, no-transform', 'Www-Authenticate': 'OAuth realm="https%3A%2F%2Fjirasimplelib.atlassian.net", OAuth realm="https%3A%2F%2Fjirasimplelib.atlassian.net"', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-Xss-Protection': '1; mode=block', 'Atl-Traceid': 'fdbe61f5d4fd47fdb78120b93b911a06', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Report-To': '{"endpoints": [{"url": "https://dz8aopenkvv6s.cloudfront.net"}], "group": "endpoint-1", "include_subdomains": true, "max_age": 600

In [2]:

def create_jira_connection(jira_url, user, api_token):
    """
    Establishes a connection to the Jira server using the provided credentials.

    Args:
        jira_url (str): The URL of the Jira server.
        username (str): The username for Jira authentication.
        api_token (str): The API token for Jira authentication.

    Returns:
        JIRA: An instance of the JIRA client.

    Raises:
        ValueError: If any of the inputs (jira_url, username, api_token) are missing.
        JIRAError: If there is an error specific to the Jira API.
        Exception: For any other exceptions that may occur.
    """
    if not (jira_url and user and api_token):
        raise ValueError("Missing username, API token, or Jira URL")

    try:
        jira = JIRA(basic_auth=(user, api_token), options={"server": jira_url})
        return jira
    except Exception as e:
        raise Exception(f"Error creating Jira connection: {e}")
    
def main():
    # Attempt to establish Jira connection
    if create_jira_connection(jira_url, user, api_token):
        print("Jira connection established successfully.")
    else:
        print("Failed to establish Jira connection.")

# Example usage in main or script execution
main()



Jira connection established successfully.


In [3]:

def create_jira_project_widget(jira):
    project_name_text = widgets.Text(description='Project Name:')
    project_key_text = widgets.Text(description='Project Key:')
    create_button = widgets.Button(description='Create Project')

    output = widgets.Output()

    def create_project(button):
        with output:
            clear_output()
            project_name = project_name_text.value
            project_key = project_key_text.value

            if not (jira and project_name and project_key):
                print("Error: Jira connection, project name, and project key must be provided.")
                return

            try:
                project = jira.create_project(project_key, project_name)
                print(f"Project '{project_name}' created successfully with key '{project_key}'.")
                return project
            except Exception as e:
                print(f"Error creating project: {e}")

    create_button.on_click(create_project)
    display(widgets.VBox([project_name_text, project_key_text, create_button, output]))
    
def create_jira_project(jira, project_name=None, project_key=None):
    if not (jira and project_name and project_key):
        raise ValueError("Jira connection, project name, and project key must be provided.")

    try:
        project = jira.create_project(project_key, project_name)
        print(f"Project '{project_name}' created successfully with key '{project_key}'.")
        return project
    except Exception as e:
        print(f"Error creating project: {e}")

def main():
    if create_jira_connection(jira_url, user, api_token):
        print("Jira connection established successfully.")
    else:
        print("Failed to establish Jira connection.")

    create_jira_project_widget(jira)

# Example usage in main or script execution
main()




Jira connection established successfully.


In [4]:

def create_story_widget(jira):
    project_key_text = widgets.Text(description='Project Key:')
    summary_text = widgets.Text(description='Summary:')
    description_text = widgets.Textarea(description='Description:')
    create_button = widgets.Button(description='Create Story')

    output = widgets.Output()

    def create_story_button_clicked(button):
        with output:
            clear_output()
            project_key = project_key_text.value.strip()
            summary = summary_text.value.strip()
            description = description_text.value.strip()

            if not (jira and project_key and summary and description):
                print("Error: Jira connection, project key, summary, and description must be provided.")
                return

            try:
                new_story = create_story(jira, project_key, summary, description)
                if new_story:
                    print(f"Story created successfully. Story Key: {new_story.key}")
            except Exception as e:
                print(f"Error creating story: {e}")

    create_button.on_click(create_story_button_clicked)
    display(widgets.VBox([project_key_text, summary_text, description_text, create_button, output]))

def create_story(jira, project_key, summary, description):
    """
    Creates a new story in Jira.

    Args:
        jira (JIRA): An authenticated JIRA client instance.
        project_key (str): The key of the project where the story will be created.
        summary (str): The summary of the story.
        description (str): The description of the story.

    Returns:
        Issue: The created Jira issue if successful, None otherwise.

    Raises:
        ValueError: If jira connection, project_key, summary, or description is not provided.
        JIRAError: If there is an error specific to the Jira API.
        Exception: For any other exceptions that may occur.
    """
    if not jira:
        logging.error("Failed to create story: Jira connection not established.")
        raise ValueError("Jira connection must be provided.")

    if not project_key:
        logging.error("Failed to create story: Project key not provided.")
        raise ValueError("Project key must be provided.")

    if not summary:
        logging.error("Failed to create story: Summary not provided.")
        raise ValueError("Summary must be provided.")

    if not description:
        logging.error("Failed to create story: Description not provided.")
        raise ValueError("Description must be provided.")

    try:
        new_story = jira.create_issue(
            project=project_key,
            summary=summary,
            description=description,
            issuetype={"name": "Task"},
        )
        logging.info(f"Story created successfully. Story Key: {new_story.key}")
        return new_story
    except JIRAError as e:
        logging.error(f"Error creating story: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error creating story: {e}")
        raise
    
def main():
    if create_jira_connection(jira_url, user, api_token):
        print("Jira connection established successfully.")
    else:
        print("Failed to establish Jira connection.")

    create_story_widget(jira)

# Example usage in main or script execution
main()




Jira connection established successfully.
